# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbbd031efd0>
├── 'a' --> tensor([[-1.2300,  0.4737, -1.7073],
│                   [-2.1459,  0.0243,  1.4678]])
└── 'x' --> <FastTreeValue 0x7fbbd031efa0>
    └── 'c' --> tensor([[ 0.9088,  0.3269,  0.6333, -1.1087],
                        [ 0.8734,  0.7349,  0.6809, -0.9307],
                        [ 2.0375, -1.0453,  0.5688, -0.5259]])

In [4]:
t.a

tensor([[-1.2300,  0.4737, -1.7073],
        [-2.1459,  0.0243,  1.4678]])

In [5]:
%timeit t.a

70.4 ns ± 2.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbbd031efd0>
├── 'a' --> tensor([[0.3829, 0.4742, 0.0154],
│                   [1.5839, 0.5724, 0.0762]])
└── 'x' --> <FastTreeValue 0x7fbbd031efa0>
    └── 'c' --> tensor([[ 0.9088,  0.3269,  0.6333, -1.1087],
                        [ 0.8734,  0.7349,  0.6809, -0.9307],
                        [ 2.0375, -1.0453,  0.5688, -0.5259]])

In [7]:
%timeit t.a = new_value

72.1 ns ± 1.06 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.2300,  0.4737, -1.7073],
               [-2.1459,  0.0243,  1.4678]]),
    x: Batch(
           c: tensor([[ 0.9088,  0.3269,  0.6333, -1.1087],
                      [ 0.8734,  0.7349,  0.6809, -0.9307],
                      [ 2.0375, -1.0453,  0.5688, -0.5259]]),
       ),
)

In [10]:
b.a

tensor([[-1.2300,  0.4737, -1.7073],
        [-2.1459,  0.0243,  1.4678]])

In [11]:
%timeit b.a

69.4 ns ± 0.947 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1954, -0.1837, -0.1268],
               [-1.7786, -0.4548, -0.2155]]),
    x: Batch(
           c: tensor([[ 0.9088,  0.3269,  0.6333, -1.1087],
                      [ 0.8734,  0.7349,  0.6809, -0.9307],
                      [ 2.0375, -1.0453,  0.5688, -0.5259]]),
       ),
)

In [13]:
%timeit b.a = new_value

615 ns ± 7.47 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.01 µs ± 9.79 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.7 µs ± 252 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

306 µs ± 17.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

296 µs ± 5.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbbd031ec10>
├── 'a' --> tensor([[[-1.2300,  0.4737, -1.7073],
│                    [-2.1459,  0.0243,  1.4678]],
│           
│                   [[-1.2300,  0.4737, -1.7073],
│                    [-2.1459,  0.0243,  1.4678]],
│           
│                   [[-1.2300,  0.4737, -1.7073],
│                    [-2.1459,  0.0243,  1.4678]],
│           
│                   [[-1.2300,  0.4737, -1.7073],
│                    [-2.1459,  0.0243,  1.4678]],
│           
│                   [[-1.2300,  0.4737, -1.7073],
│                    [-2.1459,  0.0243,  1.4678]],
│           
│                   [[-1.2300,  0.4737, -1.7073],
│                    [-2.1459,  0.0243,  1.4678]],
│           
│                   [[-1.2300,  0.4737, -1.7073],
│                    [-2.1459,  0.0243,  1.4678]],
│           
│                   [[-1.2300,  0.4737, -1.7073],
│                    [-2.1459,  0.0243,  1.4678]]])
└── 'x' --> <FastTreeValue 0x7fbb2cc8bfa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.9 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fbb1bbb6b80>
├── 'a' --> tensor([[-1.2300,  0.4737, -1.7073],
│                   [-2.1459,  0.0243,  1.4678],
│                   [-1.2300,  0.4737, -1.7073],
│                   [-2.1459,  0.0243,  1.4678],
│                   [-1.2300,  0.4737, -1.7073],
│                   [-2.1459,  0.0243,  1.4678],
│                   [-1.2300,  0.4737, -1.7073],
│                   [-2.1459,  0.0243,  1.4678],
│                   [-1.2300,  0.4737, -1.7073],
│                   [-2.1459,  0.0243,  1.4678],
│                   [-1.2300,  0.4737, -1.7073],
│                   [-2.1459,  0.0243,  1.4678],
│                   [-1.2300,  0.4737, -1.7073],
│                   [-2.1459,  0.0243,  1.4678],
│                   [-1.2300,  0.4737, -1.7073],
│                   [-2.1459,  0.0243,  1.4678]])
└── 'x' --> <FastTreeValue 0x7fbb1b5e4f40>
    └── 'c' --> tensor([[ 0.9088,  0.3269,  0.6333, -1.1087],
                        [ 0.8734,  0.7349,  0.6809, -0.9307],
                 

In [23]:
%timeit t_cat(trees)

43 µs ± 621 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

85.3 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.9088,  0.3269,  0.6333, -1.1087],
                       [ 0.8734,  0.7349,  0.6809, -0.9307],
                       [ 2.0375, -1.0453,  0.5688, -0.5259]],
              
                      [[ 0.9088,  0.3269,  0.6333, -1.1087],
                       [ 0.8734,  0.7349,  0.6809, -0.9307],
                       [ 2.0375, -1.0453,  0.5688, -0.5259]],
              
                      [[ 0.9088,  0.3269,  0.6333, -1.1087],
                       [ 0.8734,  0.7349,  0.6809, -0.9307],
                       [ 2.0375, -1.0453,  0.5688, -0.5259]],
              
                      [[ 0.9088,  0.3269,  0.6333, -1.1087],
                       [ 0.8734,  0.7349,  0.6809, -0.9307],
                       [ 2.0375, -1.0453,  0.5688, -0.5259]],
              
                      [[ 0.9088,  0.3269,  0.6333, -1.1087],
                       [ 0.8734,  0.7349,  0.6809, -0.9307],
                       [ 2.0375, -1.0453,  0.5688, -0.5259]],

In [26]:
%timeit Batch.stack(batches)

106 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.9088,  0.3269,  0.6333, -1.1087],
                      [ 0.8734,  0.7349,  0.6809, -0.9307],
                      [ 2.0375, -1.0453,  0.5688, -0.5259],
                      [ 0.9088,  0.3269,  0.6333, -1.1087],
                      [ 0.8734,  0.7349,  0.6809, -0.9307],
                      [ 2.0375, -1.0453,  0.5688, -0.5259],
                      [ 0.9088,  0.3269,  0.6333, -1.1087],
                      [ 0.8734,  0.7349,  0.6809, -0.9307],
                      [ 2.0375, -1.0453,  0.5688, -0.5259],
                      [ 0.9088,  0.3269,  0.6333, -1.1087],
                      [ 0.8734,  0.7349,  0.6809, -0.9307],
                      [ 2.0375, -1.0453,  0.5688, -0.5259],
                      [ 0.9088,  0.3269,  0.6333, -1.1087],
                      [ 0.8734,  0.7349,  0.6809, -0.9307],
                      [ 2.0375, -1.0453,  0.5688, -0.5259],
                      [ 0.9088,  0.3269,  0.6333, -1.1087],
                   

In [28]:
%timeit Batch.cat(batches)

244 µs ± 9.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

608 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
